In [1]:
import yaml
import src.utils as utils
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as scs
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

## 1. Load All Preprocessed Dataset

In [3]:
x_train = utils.deserialize_data("data/processed/x_train_prep.pkl")
y_train = utils.deserialize_data("data/processed/y_train_prep.pkl")

x_valid = utils.deserialize_data("data/processed/x_valid_prep.pkl")
y_valid = utils.deserialize_data("data/processed/y_valid_prep.pkl")

x_test = utils.deserialize_data("data/processed/x_test_prep.pkl")
y_test = utils.deserialize_data("data/processed/y_test_prep.pkl")

In [4]:
train_set = utils.combine_dataframe([x_train, y_train], axis=1)
valid_set = utils.combine_dataframe([x_valid, y_valid], axis=1)
test_set = utils.combine_dataframe([x_test, y_test], axis=1)

In [15]:
params = utils.load_params("config/params.yaml")

## 2. Outliers Removal

In [5]:
def quartile_based_outliers_removal(target_dataframe, columns_name):
    target_dataframe = target_dataframe.copy(deep=True)

    for column_name in columns_name:
        q1 = target_dataframe[column_name].quantile(0.25)
        q3 = target_dataframe[column_name].quantile(0.75)
        iqr = q3 - q1

        lower_limit = q1 - 1.5 * iqr
        upper_limit = q3 + 1.5 * iqr

        target_dataframe = target_dataframe[(target_dataframe[column_name] > lower_limit) & (target_dataframe[column_name] < upper_limit)]
    
    return target_dataframe     


In [9]:
train_set_rmout = quartile_based_outliers_removal(train_set, params["predictors_feature_engineered"])

In [10]:
train_set.describe()

,DKI1 (Bunderan HI),DKI2 (Kelapa Gading),DKI3 (Jagakarsa),DKI4 (Lubang Buaya),DKI5 (Kebon Jeruk) Jakarta Barat,pm10,pm25,so2,co,o3,no2,categori
count,1269.000000,1269.000000,1269.000000,1269.000000,1269.000000,1269.000000,1269.000000,1269.000000,1269.000000,1269.000000,1269.000000,1269.000000
mean,0.198582,0.197794,0.204098,0.201734,0.197794,52.613869,79.198582,35.289204,11.631206,31.662727,19.402679,0.895981
std,0.399090,0.398493,0.403200,0.401453,0.398493,14.840764,24.329396,12.227305,4.920135,14.356701,9.071623,0.305405
min,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,13.000000,3.000000,2.000000,7.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,44.000000,64.000000,26.000000,9.000000,22.000000,13.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,54.000000,79.000000,35.000000,11.000000,29.000000,18.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,62.000000,93.000000,43.000000,14.000000,37.000000,25.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,100.000000,174.000000,82.000000,44.000000,151.000000,65.000000,1.000000


In [11]:
train_set_rmout.describe()

,DKI1 (Bunderan HI),DKI2 (Kelapa Gading),DKI3 (Jagakarsa),DKI4 (Lubang Buaya),DKI5 (Kebon Jeruk) Jakarta Barat,pm10,pm25,so2,co,o3,no2,categori
count,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000
mean,0.206393,0.168037,0.222831,0.200000,0.202740,51.610046,77.344292,35.223744,10.865753,29.604566,18.712329,0.892237
std,0.404901,0.374070,0.416336,0.400183,0.402224,13.734239,21.964097,11.466196,3.688669,10.875974,7.907624,0.310222
min,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,21.000000,3.000000,2.000000,9.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,44.000000,63.000000,27.000000,8.000000,22.000000,13.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,53.000000,79.000000,35.000000,11.000000,28.000000,18.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,61.000000,91.000000,43.000000,13.000000,35.000000,24.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,87.000000,134.000000,67.000000,21.000000,59.000000,40.000000,1.000000


In [12]:
x_train, y_train = utils.split_predictor_target(train_set_rmout, params)

In [13]:
# here we have:
# 1. train_set variable, untouched one
# 2. train_set_rmout, with outlier removed

## 3. Feature Scaling

#### 3.1. Min-Max Normalization

In [14]:
def minmax_norm(target_dataframe, columns_name):
    target_dataframe = target_dataframe.copy(deep=True)

    minmax_object = dict()
    for column_name in columns_name:
        data = np.array(target_dataframe[column_name]).reshape(-1, 1)
        minmax_object["mms_{}".format(column_name)] = MinMaxScaler()
        data = minmax_object["mms_{}".format(column_name)].fit_transform(data)
        target_dataframe[column_name] = data

    return minmax_object, target_dataframe

In [19]:
train_set_minmax, minmax_scaler_object = minmax_norm(train_set, params["predictors_feature_engineered"])

In [20]:
train_set_rmout_minmax, rmout_minmax_scaler_object = minmax_norm(train_set_rmout, params["predictors_feature_engineered"])

In [21]:
# until this part, we have 4 train data:
# 1. train_set
# 2. train_set_outliers_removed
# 3. train_set_minmax
# 4. train_set_outliers_removed_minmax

# also we have 2 more object:
# 1. minmax scaler object for dataset train_set, it's called minmax_scaler_object
# 2. minmax scaler object for dataset train_set_rmout, it's called rmout_minmax_scaler_object

#### 3.2. Standardization